In [1]:
!pip3 install --upgrade alexnet_pytorch

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
import torch 
import torchvision
import torchvision.transforms as transforms
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import time
from alexnet_pytorch import AlexNet
import torch.optim as optim

In [3]:
torch.manual_seed(0)


In [ ]:
transform = transforms.Compose([transforms.ToTensor(),transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225))])

trainset = torchvision.datasets.STL10(root="./data",split='train',transform=transform,download=True)



testset = torchvision.datasets.STL10(root='./data', split='test',download=True, transform=transform)


  0%|          | 0/2640397119 [00:00<?, ?it/s]

In [ ]:
trainloader = torch.utils.data.DataLoader(trainset, batch_size=8,shuffle=True, num_workers=2)
testloader = torch.utils.data.DataLoader(testset, batch_size=8,shuffle=False, num_workers=2)
classes = ('airplane', 'bird', 'car', 'cat', 'deer', 'dog', 'horse', 'monkey', 'ship', 'truck')

In [ ]:
AlexNet_model = torch.hub.load('pytorch/vision:v0.6.0', 'alexnet', pretrained=True)

In [ ]:
AlexNet_model.eval()

In [ ]:
AlexNet_model.classifier[4] = nn.Linear(4096,1024)
AlexNet_model.classifier[6] = nn.Linear(1024,10)

In [ ]:
AlexNet_model.eval()

In [ ]:
import torch.optim as optim

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(AlexNet_model.parameters(), lr=0.0009,momentum=0.85)
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [ ]:
AlexNet_model.to(device)
criterion.to(device)

In [ ]:
# i, data in enumerate(trainloader,0)
# print((i))

In [ ]:
start_time = time.time()
for epoch in range(10):  # loop over the dataset multiple times

    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        # get the inputs; data is a list of [inputs, labels]
        inputs, labels = data[0].to(device), data[1].to(device)

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = AlexNet_model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()
        if i % 312 == 311:    # print every 1000 mini-batches
            print('[%d, %5d] loss: %.3f' %
                  (epoch + 1, i + 1, running_loss / 312))
            # print(loss.item())
            running_loss = 0.0

print('Finished Training')
print("Time taken to train is %s seconds" % (time.time() - start_time))

In [ ]:
correct = 0
total = 0
with torch.no_grad():
    for data in testloader:
        images, labels = data
        images,labels = images.to(device),labels.to(device)
        outputs = AlexNet_model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print('Accuracy of the network on the 10000 test images: %d %%' % (
    100 * correct / total))

In [ ]:

def make_data(dataloader):
    train_features,train_labels = [],[]
    with torch.no_grad():
        for data in dataloader:
            images, labels = data
            images,labels = images.to(device),labels.to(device)
            output = AlexNet_model(images)
            train_features.extend(output)
            train_labels.extend(labels)
    return train_features,train_labels

In [ ]:
train_features,train_labels = make_data(trainloader)

In [ ]:
test_features,test_labels = make_data(testloader)

In [ ]:
AlexNet_model=torch.nn.Sequential(AlexNet_model)
AlexNet_model=AlexNet_model[:-1]

In [ ]:
device1=torch.device('cpu')
training_data = [t.to(device1).numpy() for t in train_features]
labels = [t.to(device1).numpy() for t in train_labels]
testing_data = [t.to(device1).numpy() for t in train_features]
test_labels = [t.to(device1).numpy() for t in train_labels]

In [ ]:
import pandas as pd
training_data,labels = pd.DataFrame(training_data),pd.DataFrame(labels)
testing_data,testing_labels = pd.DataFrame(testing_data),pd.DataFrame(test_labels)

In [ ]:
train = pd.concat([training_data,testing_data])
y = pd.concat([labels,testing_labels])

In [ ]:
from sklearn.model_selection import train_test_split

X_train,X_test,y_train,y_test = train_test_split(train,y,shuffle=True,random_state=2021,test_size=0.1)

In [ ]:
from sklearn.svm import SVC

svc = SVC(kernel='linear')
svc.fit(X_train,y_train)

In [ ]:
from sklearn.metrics import accuracy_score

y_pred = svc.predict(X_test)
print(accuracy_score(y_test,y_pred))

In [ ]:
from sklearn.linear_model import LogisticRegression

lr = LogisticRegression(max_iter = 1000,l1_ratio=0)
lr.fit(X_train,y_train)

In [ ]:
from sklearn.metrics import accuracy_score

y_pred = lr.predict(X_test)
print(accuracy_score(y_test,y_pred))